# Business Case #5 - Retail - Demand Forecasting

## Authors:
#### Débora Santos (m20200748),Pedro Henrique Medeiros (m20200742), Rebeca Pinheiro (m20201096)

#### Group D - D4B Consulting

In [ ]:
!pip install lightgbm

In [1]:
#IMPORT LIBRARIES
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import seaborn as sns
from datetime import datetime
from itertools import product
from math import ceil
%matplotlib inline 
from collections import Counter
import matplotlib.cm as cm


#Models
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error, median_absolute_error
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor

# Use garbage collection to minimise memory usage
import gc

import warnings
warnings.filterwarnings("ignore")

# Seeting seaborn style
sns.set()

In [2]:
def downcast1(df, verbose=True):
    
    """
    Funciton to reduce the memory used of a particular dataframe by downcasting to a less memory-intensive data type.
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('{:.1f}% compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
#import dataset in csv
df_forecast = pd.read_csv('df_demand_complete.csv')

In [4]:
#reduce memory usage
df_forecast = downcast1(df_forecast)
df_forecast.info()

65.3% compressed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120361747 entries, 0 to 120361746
Data columns (total 22 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   Year_Week                               int32  
 1   Store_ID                                int16  
 2   Product_ID                              int16  
 3   Quantity                                int8   
 4   Product_ID_Quantity_mean                float32
 5   Product_ID_Store_ID_Quantity_mean       int16  
 6   Store_ID_Quantity_mean                  float32
 7   Quantity_lag1                           int8   
 8   Quantity_lag2                           int8   
 9   Quantity_lag3                           int8   
 10  Product_ID_Quantity_mean_lag1           float32
 11  Product_ID_Quantity_mean_lag2           float32
 12  Product_ID_Quantity_mean_lag3           float32
 13  Product_ID_Store_ID_Quantity_mean_lag1  int8   
 14  Product_ID_St

In [5]:
df_forecast.head()

Year_Week  Store_ID  Product_ID  Quantity  Product_ID_Quantity_mean  \
0     201728         1        1000         2                  1.828851   
1     201728         1        1001         0                  0.056235   
2     201728         1        1002         4                  1.303178   
3     201728         1        1003         0                  2.026895   
4     201728         1        1004         0                  2.009780   

   Product_ID_Store_ID_Quantity_mean  Store_ID_Quantity_mean  Quantity_lag1  \
0                                  2                2.163561              0   
1                                  0                2.163561              0   
2                                  4                2.163561              2   
3                                  0                2.163561              0   
4                                  0                2.163561              0   

   Quantity_lag2  Quantity_lag3  ...  Product_ID_Quantity_mean_lag3  \
0              0              0  ...                       1.687042   
1              0              0  ...                       0.034230   
2              5              4  ...                       1.293398   
3              0              0  ...                       1.762836   
4              2              2  ...                       1.684597   

   Product_ID_Store_ID_Quantity_mean_lag1  \
0                                       0   
1                                       0   
2                                       2   
3                                       0   
4                                       0   

   Product_ID_Store_ID_Quantity_mean_lag2  \
0                                       0   
1                                       0   
2                                       5   
3                                       0   
4                                       2   

   Product_ID_Store_ID_Quantity_mean_lag3  Store_ID_Quantity_mean_lag1  \
0                                       0                     1.985345   
1                                       0                     1.985345   
2                                       4                     1.985345   
3                                       0                     1.985345   
4                                       2                     1.985345   

   Store_ID_Quantity_mean_lag2  Store_ID_Quantity_mean_lag3  \
0                     1.992302                     1.791754   
1                     1.992302                     1.791754   
2                     1.992302                     1.791754   
3                     1.992302                     1.791754   
4                     1.992302                     1.791754   

   Quantity_lag3_mean  lag_grad_1  lag_grad_2  
0            0.000000         0.0        0.00  
1            0.000000         0.0        0.00  
2            3.666667         0.4        1.25  
3            0.000000         0.0        0.00  
4            1.333333         0.0        1.00  

[5 rows x 22 columns]

### Split the data

In [6]:
#Create train, val and test data spliting by last 6 weeks to test set, more 6 weeks to validation set 
train = df_forecast[df_forecast['Year_Week']<201932]

In [7]:
val = df_forecast[df_forecast['Year_Week']>=201932]
val = val[val['Year_Week']<201938]

In [8]:
test = df_forecast[df_forecast['Year_Week']>=201938]

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111647593 entries, 0 to 115659473
Data columns (total 22 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   Year_Week                               int32  
 1   Store_ID                                int16  
 2   Product_ID                              int16  
 3   Quantity                                int8   
 4   Product_ID_Quantity_mean                float32
 5   Product_ID_Store_ID_Quantity_mean       int16  
 6   Store_ID_Quantity_mean                  float32
 7   Quantity_lag1                           int8   
 8   Quantity_lag2                           int8   
 9   Quantity_lag3                           int8   
 10  Product_ID_Quantity_mean_lag1           float32
 11  Product_ID_Quantity_mean_lag2           float32
 12  Product_ID_Quantity_mean_lag3           float32
 13  Product_ID_Store_ID_Quantity_mean_lag1  int8   
 14  Product_ID_Store_ID_Quantity_m

In [10]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4011881 entries, 14602527 to 105285597
Data columns (total 22 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   Year_Week                               int32  
 1   Store_ID                                int16  
 2   Product_ID                              int16  
 3   Quantity                                int8   
 4   Product_ID_Quantity_mean                float32
 5   Product_ID_Store_ID_Quantity_mean       int16  
 6   Store_ID_Quantity_mean                  float32
 7   Quantity_lag1                           int8   
 8   Quantity_lag2                           int8   
 9   Quantity_lag3                           int8   
 10  Product_ID_Quantity_mean_lag1           float32
 11  Product_ID_Quantity_mean_lag2           float32
 12  Product_ID_Quantity_mean_lag3           float32
 13  Product_ID_Store_ID_Quantity_mean_lag1  int8   
 14  Product_ID_Store_ID_Quant

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4702273 entries, 115659474 to 120361746
Data columns (total 22 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   Year_Week                               int32  
 1   Store_ID                                int16  
 2   Product_ID                              int16  
 3   Quantity                                int8   
 4   Product_ID_Quantity_mean                float32
 5   Product_ID_Store_ID_Quantity_mean       int16  
 6   Store_ID_Quantity_mean                  float32
 7   Quantity_lag1                           int8   
 8   Quantity_lag2                           int8   
 9   Quantity_lag3                           int8   
 10  Product_ID_Quantity_mean_lag1           float32
 11  Product_ID_Quantity_mean_lag2           float32
 12  Product_ID_Quantity_mean_lag3           float32
 13  Product_ID_Store_ID_Quantity_mean_lag1  int8   
 14  Product_ID_Store_ID_Quan

In [12]:
#Remove target variable and save it in separate
X_train = train.drop('Quantity', axis=1)
y_train = train['Quantity']
X_val = val.drop('Quantity', axis=1)
y_val = val['Quantity']

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111647593 entries, 0 to 115659473
Data columns (total 21 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   Year_Week                               int32  
 1   Store_ID                                int16  
 2   Product_ID                              int16  
 3   Product_ID_Quantity_mean                float32
 4   Product_ID_Store_ID_Quantity_mean       int16  
 5   Store_ID_Quantity_mean                  float32
 6   Quantity_lag1                           int8   
 7   Quantity_lag2                           int8   
 8   Quantity_lag3                           int8   
 9   Product_ID_Quantity_mean_lag1           float32
 10  Product_ID_Quantity_mean_lag2           float32
 11  Product_ID_Quantity_mean_lag3           float32
 12  Product_ID_Store_ID_Quantity_mean_lag1  int8   
 13  Product_ID_Store_ID_Quantity_mean_lag2  int8   
 14  Product_ID_Store_ID_Quantity_m

In [14]:
X_train = downcast1(X_train)

0.0% compressed


In [15]:
del df_forecast
gc.collect()

40

### Models

#### DECISION TREE - REGRESSOR

In [ ]:
#Run the model with parameters defined 
#Get the R2
model_DT = DecisionTreeRegressor(max_depth=3, min_samples_split=0.5)
model_DT.fit(X_train, y_train)
y_pred_DT = model_DT.predict(X_val)
DT_score = model_DT.score(X_val, y_val)
DT_score

In [ ]:
#Get the adjusted R2
r2 = r2_score(y_val, y_pred_DT)
n = len(y_val)
p = len(X_train.columns)

def adj_r2 (r2,n,p):
    return 1-(1-r2)*(n-1)/(n-p-1)

adj_r2(r2,n,p)

In [ ]:
#Get MSE
mean_squared_error(y_val, y_pred_DT, squared = True)

#### LIGHT GBM

#### Light GBM is a gradient boosting that uses tree based learning algorithm. This algorithm is very good to work with very big dataset (our case). LGBM has a high speed and it takes lower memory to run.  In LGBM the tree grows vertically while the other algorithms the tree grows horizontally. 


In [16]:
#Create a function to apply the algorithm LIGHT GBM
def build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features):
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    model = lgb.train(params=params, train_set=lgb_train, valid_sets=(lgb_train, lgb_val), verbose_eval=50,
                     categorical_feature=cat_features)
    return model

In [17]:
#Define the parameters of algorith
params = {
    'objective': 'rmse',
    'metric': 'rmse',
    'num_leaves': 500,
    'min_data_in_leaf':10,
    'feature_fraction':0.7,
    'learning_rate': 0.01,
    'early_stopping_rounds':10,
    'seed': 1
}

In [18]:
#designating the categorical features which should be focused on
cat_features = ['Year_Week','Store_ID', 'Product_ID']

In [19]:
#Run the model with parameters defined 
lgb_model = build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features)

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 10.773544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5290
[LightGBM] [Info] Number of data points in the train set: 111647593, number of used features: 21
[LightGBM] [Info] Start training from score 1.335993
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	training's rmse: 2.31338	valid_1's rmse: 2.17545
[100]	training's rmse: 1.4578	valid_1's rmse: 1.37191
Did not meet early stopping. Best iteration is:
[100]	training's rmse: 1.4578	valid_1's rmse: 1.37191


In [20]:
#Get the R2
y_pred_lgb = lgb_model.predict(X_val)
r2_lgb = r2_score(y_val, y_pred_lgb)
r2_lgb

0.8452213154615835

In [22]:
#Get the adjusted R2
n = len(y_val)
p = len(X_train.columns)

def adj_r2 (r2,n,p):
    return 1-(1-r2)*(n-1)/(n-p-1)

adj_r2(r2_lgb,n,p)

0.8452205052754889

In [23]:
#Get MSE
mean_squared_error(y_val, y_pred_lgb, squared = True)

1.8821383494802897

#### STORE RESULTS

In [ ]:
X_train_new = X_train.copy()
X_train_new

In [ ]:
X_val_new = X_val.copy()
X_val_new 

In [ ]:
X_train_new = downcast1(X_train_new)

In [ ]:
#Create the columns to store the results LGBM 
X_train_new['lgb_pred'] = lgb_model.predict(X_train).clip(0,20)
X_train_new['target'] = y_train
X_train_new['sq_err_lgb'] = (X_train_new['lgb_pred']-X_train_new['target'])**2

In [ ]:
#Create the columns to store the results LGBM 
X_val_new['lgb_pred'] = lgb_model.predict(X_val).clip(0,20)
X_val_new['target'] = y_val
X_val_new['sq_err_lgb'] = (X_val_new['lgb_pred']-X_val_new['target'])**2

In [ ]:
#Create the columns to store the results DT
X_train_new['DT_pred'] = model_DT_samp.predict(X_train).clip(0,20)
X_train_new['sq_err_DT'] = (X_train_new['DT_pred']-X_train_new['target'])**2

In [ ]:
#Create the columns to store the results DT
X_val_new['DT_pred'] = model_DT_samp.predict(X_val).clip(0,20)
X_val_new['sq_err_DT'] = (X_val_new['DT_pred']-X_val_new['target'])**2

#### APPLY ON TEST SET 

In [ ]:
#import dataset in csv
test_original = pd.read_csv('teste.csv')

In [ ]:
#TEST SET ORIGINAL
test_original.head()

In [ ]:
#TEST SET MODIFIED (WITHZERO QUANTITIES AND PREPROCESSED)
X_test = test.drop('Quantity',axis = 1 )
y_test = test['Quantity']

In [ ]:
#DEFINE FEATURES TO MERGE TEST DATASETS
index_feats = ["Year_Week", "Store_ID", "Product_ID"]

In [ ]:
#MERGE DATASETS
X_test_new =pd.merge(X_test, test_original, on=index_feats, how="left")
X_test_new 

In [ ]:
#STORE RESULTS LGBM
X_test_new['lgb_pred'] = lgb_model.predict(X_test).clip(0,20)
X_test_new['target'] = X_test_new['Quantity']
X_test_new['sq_err_lgb'] = (X_test_new['lgb_pred']-X_test_new['target'])**2

In [ ]:
#STORE RESULTS DT
X_test_new['DT_pred'] = model_DT_samp.predict(X_test).clip(0,20)
X_test_new['sq_err_DT'] = (X_test_new['DT_pred']-X_test_new['target'])**2

In [ ]:
#GET RMSE
mean_squared_error(X_test_new['target'], X_test_new['lgb_pred'], squared = True)

In [ ]:
#GET MSE
mean_squared_error(X_test_new['target'], X_test_new['DT_pred'], squared = True)

In [ ]:
#CREATE A COLUMN TO IDENTIFY WITCH PART OF DATASET 
X_train_new["dataset"] = 'Train'
X_val_new["dataset"] = 'Validation'
X_test_new["dataset"] = 'Test'

In [ ]:
#Create a function to calculate WAPE
def wmape_gr(df_in, st_actual, st_forecast):
    # we take two series and calculate an output a wmape from it

    # make a series called mape
    se_mape = abs(df_in[st_actual] - df_in[st_forecast]) / df_in[st_actual]

    # get a float of the sum of the actual
    ft_actual_sum = df_in[st_actual].sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = df_in[st_actual] * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast

In [ ]:
#Calculate wape to lbgm
df_gr_train_LGB = X_train_new.groupby(['Store_ID', 'Product_ID','dataset']).apply(wmape_gr,'target','lgb_pred')

In [ ]:
df_gr_train_LGB.head()

In [ ]:
#Calculate wape to DT
df_gr_train_DT = X_train_new.groupby(['Store_ID', 'Product_ID','dataset']).apply(wmape_gr,'target','DT_pred')

In [ ]:
#Reset index
df_gr_train_LGB = df_gr_train_LGB.reset_index()
#Rename aggregate collumn
df_gr_train_LGB = df_gr_train_LGB.rename(columns={0:'wape_lgb'})
#Replace nams by 0
df_gr_train_LGB = df_gr_train_LGB.fillna(0)
df_gr_train_LGB

In [ ]:
#Reset index
df_gr_train_DT = df_gr_train_DT.reset_index()
#Rename aggregate collumn
df_gr_train_DT = df_gr_train_DT.rename(columns={0:'wape_DT'})
#Replace nams by 0
df_gr_train_DT = df_gr_train_DT.fillna(0)
df_gr_train_DT

In [ ]:
#merge wape DT and WAPE LGBM
df_gr_train = pd.merge(df_gr_train_DT, df_gr_train_LGB, on= ['Store_ID', 'Product_ID','dataset'], how="left")

In [ ]:
del df_gr_train_DT
gc.collect()

In [ ]:
del df_gr_train_LGB
gc.collect()

In [ ]:
#Calculate wape to lbgm
df_gr_val_LGB = X_val_new.groupby(['Store_ID', 'Product_ID','dataset']).apply(wmape_gr,'target','lgb_pred')

In [ ]:
#Calculate wape to DT
df_gr_val_DT = X_val_new.groupby(['Store_ID', 'Product_ID','dataset']).apply(wmape_gr,'target','DT_pred')

In [ ]:
df_gr_val_LGB = df_gr_val_LGB.reset_index()
df_gr_val_LGB = df_gr_val_LGB.rename(columns={0:'wape_lgb'})
df_gr_val_LGB = df_gr_val_LGB.fillna(0)
df_gr_val_LGB

In [ ]:
df_gr_val_DT = df_gr_val_DT.reset_index()
df_gr_val_DT = df_gr_val_DT.rename(columns={0:'wape_DT'})
df_gr_val_DT = df_gr_val_DT.fillna(0)
df_gr_val_DT

In [ ]:
#Merge results
df_gr_val = pd.merge(df_gr_val_DT, df_gr_val_LGB, on= ['Store_ID', 'Product_ID','dataset'], how="left")

In [ ]:
#Calculate wape to lbgm
df_gr_test_LGB = X_test_new.groupby(['Store_ID', 'Product_ID','dataset']).apply(wmape_gr,'target','lgb_pred')

In [ ]:
#Calculate wape to DT
df_gr_test_DT = X_test_new.groupby(['Store_ID', 'Product_ID','dataset']).apply(wmape_gr,'target','DT_pred')

In [ ]:
df_gr_test_LGB = df_gr_test_LGB.reset_index()
df_gr_test_LGB = df_gr_test_LGB.rename(columns={0:'wape_lgb'})
df_gr_test_LGB = df_gr_test_LGB.fillna(0)
df_gr_test_LGB

In [ ]:
df_gr_test_DT = df_gr_test_DT.reset_index()
df_gr_test_DT = df_gr_test_DT.rename(columns={0:'wape_DT'})
df_gr_test_DT = df_gr_test_DT.fillna(0)
df_gr_test_DT

In [ ]:
#Merge results
df_gr_test = pd.merge(df_gr_test_DT, df_gr_test_LGB, on= ['Store_ID', 'Product_ID','dataset'], how="left")

In [ ]:
#Append all in one dataframe
df_gr_all = df_gr_train.append(df_gr_val)
df_gr_all = df_gr_all.append(df_gr_test)

In [ ]:
data = X_train_new[['Year_Week', 'Store_ID', 'Product_ID', 'lgb_pred', 'target', 'sq_err_lgb','DT_pred','sq_err_DT','dataset']]

In [ ]:
data.head()

In [ ]:
data_val = X_val_new[['Year_Week', 'Store_ID', 'Product_ID', 'lgb_pred', 'target', 'sq_err_lgb','DT_pred','sq_err_DT','dataset']]

In [ ]:
data_test = X_test_new[['Year_Week', 'Store_ID', 'Product_ID', 'lgb_pred', 'target', 'sq_err_lgb','DT_pred','sq_err_DT','dataset']]

In [ ]:
#Append all in one dataframe
data_all = data.append(data_val)
data_all = data_all.append(data_test)

In [ ]:
data_all.to_csv(os.path.join("df_demand_final.csv"), index=False)

In [ ]:
df_gr_all.to_csv(os.path.join("df_demand_wape.csv"), index=False)